<a href="https://colab.research.google.com/github/samans98/Predict-New-Medicines-with-BELKA/blob/main/belka_PepCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit
!pip install tensorflow
!pip install duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 49.1 MB/s eta 0:00:00


In [ ]:
import gc
import os
import pickle
import random
import joblib
import duckdb
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [ ]:
train_path = '/content/drive/MyDrive/Kaggle/train.parquet'
test_path = '/content/drive/MyDrive/Kaggle/test.parquet'

In [ ]:
con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 250000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 250000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,107554830,O=C(N[C@@H](Cc1ccsc1)C(=O)O)OCC1c2ccccc2-c2ccc...,NC[C@]1(CO)COC[C@H]2CCCN21,Cl.Cl.NCCNC(=O)c1cnccn1,O=C(NCCNc1nc(NC[C@]2(CO)COC[C@H]3CCCN32)nc(N[C...,BRD4,0
1,132713058,O=C(N[C@H]1CC[C@H](C(=O)O)CC1)OCC1c2ccccc2-c2c...,Nc1ccc(CC2COC(=O)N2)cc1,Cn1nnc(N)n1,Cn1nnc(Nc2nc(Nc3ccc(CC4COC(=O)N4)cc3)nc(N[C@H]...,BRD4,0
2,25327674,CN(C(=O)OCC1c2ccccc2-c2ccccc21)[C@@H](CC1CCCCC...,COc1ccc(N)cn1,Cl.NCCNC(N)=O,COc1ccc(Nc2nc(NCCNC(N)=O)nc(N(C)[C@@H](CC3CCCC...,BRD4,0
3,282600873,O=C(O)[C@@H]1CSCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cc1cnc(N)s1,COc1ccc(N)c(Cl)c1,COc1ccc(Nc2nc(Nc3ncc(C)s3)nc(N3CSC[C@H]3C(=O)N...,BRD4,0
4,127971695,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Cl.Cl.NCc1cncc(F)c1,COCc1ccccc1CN,COCc1ccccc1CNc1nc(NCc2cncc(F)c2)nc(N[C@H](Cc2c...,sEH,0


In [ ]:
train = df.copy()
train = train.drop(columns=['id','buildingblock1_smiles','buildingblock2_smiles','buildingblock3_smiles'])
train.head()

,molecule_smiles,protein_name,binds
0,O=C(NCCNc1nc(NC[C@]2(CO)COC[C@H]3CCCN32)nc(N[C...,BRD4,0
1,Cn1nnc(Nc2nc(Nc3ccc(CC4COC(=O)N4)cc3)nc(N[C@H]...,BRD4,0
2,COc1ccc(Nc2nc(NCCNC(N)=O)nc(N(C)[C@@H](CC3CCCC...,BRD4,0
3,COc1ccc(Nc2nc(Nc3ncc(C)s3)nc(N3CSC[C@H]3C(=O)N...,BRD4,0
4,COCc1ccccc1CNc1nc(NCc2cncc(F)c2)nc(N[C@H](Cc2c...,sEH,0


In [ ]:
columns = ['drug_smiles','protein_sequence','binding']
train.columns = columns
train.head()

,drug_smiles,protein_sequence,binding
0,O=C(NCCNc1nc(NC[C@]2(CO)COC[C@H]3CCCN32)nc(N[C...,BRD4,0
1,Cn1nnc(Nc2nc(Nc3ccc(CC4COC(=O)N4)cc3)nc(N[C@H]...,BRD4,0
2,COc1ccc(Nc2nc(NCCNC(N)=O)nc(N(C)[C@@H](CC3CCCC...,BRD4,0
3,COc1ccc(Nc2nc(Nc3ncc(C)s3)nc(N3CSC[C@H]3C(=O)N...,BRD4,0
4,COCc1ccccc1CNc1nc(NCc2cncc(F)c2)nc(N[C@H](Cc2c...,sEH,0


In [ ]:
#amino acid sequence sourced from uniprot
BRD4_seq = "MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPVDAVKLNLPDYYKIIKTPMDMGTIKKRLENNYYWNAQECIQDFNTMFTNCYIYNKPGDDIVLMAEALEKLFLQKINELPTEETEIMIVQAKGRGRGRKETGTAKPGVSTVPNTTQASTPPQTQTPQPNPPPVQATPHPFPAVTPDLIVQTPVMTVVPPQPLQTPPPVPPQPQPPPAPAPQPVQSHPPIIAATPQPVKTKKGVKRKADTTTPTTIDPIHEPPSLPPEPKTTKLGQRRESSRPVKPPKKDVPDSQQHPAPEKSSKVSEQLKCCSGILKEMFAKKHAAYAWPFYKPVDVEALGLHDYCDIIKHPMDMSTIKSKLEAREYRDAQEFGADVRLMFSNCYKYNPPDHEVVAMARKLQDVFEMRFAKMPDEPEEPVVAVSSPAVPPPTKVVAPPSSSDSSSDSSSDSDSSTDDSEEERAQRLAELQEQLKAVHEQLAALSQPQQNKPKKKEKDKKEKKKEKHKRKEEVEENKKSKAKEPPPKKTKKNNSSNSNVSKKEPAPMKSKPPPTYESEEEDKCKPMSYEEKRQLSLDINKLPGEKLGRVVHIIQSREPSLKNSNPDEIEIDFETLKPSTLRELERYVTSCLRKKRKPQAEKVDVIAGSSKMKGFSSSESESSSESSSSDSEDSETEMAPKSKKKGHPGREQKKHHHHHHQQMQQAPAPVPQQPPPPPQQPPPPPPPQQQQQPPPPPPPPSMPQQAAPAMKSSPPPFIATQVPVLEPQLPGSVFDPIGHFTQPILHLPQPELPPHLPQPPEHSTPPHLNQHAVVSPPALHNALPQQPSRPSNRAAALPPKPARPPAVSPALTQTPLLPQPPMAQPPQVLLEDEEPPAPPLTSMQMQLYLQQLQKVQPPTPLLPSVKVQSQPPPPLPPPPHPSVQQQLQQQPPPPPPPQPQPPPQQQHQPPPRPVHLQPMQFSTHIQQPPPPQGQQPPHPPPGQQPPPPQPAKPQQVIQHHHSPRHHKSDPYSTGHLREAPSPLMIHSPQMSQFQSLTHQSPPQQNVQPKKQELRAASVVQPQPLVVVKEEKIHSPIIRSEPFSPSLRPEPPKHPESIKAPVHLPQRPEMKPVDVGRPVIRPPEQNAPPPGAPDKDKQKQEPKTPVAPKKDLKIKNMGSWASLVQKHPTTPSSTAKSSSDSFEQFRRAAREKEEREKALKAQAEHAEKEKERLRQERMRSREDEDALEQARRAHEEARRRQEQQQQQRQEQQQQQQQQAAAVAAAATPQAQSSQPQSMLDQQRELARKREQERRRREAMAATIDMNFQSDLLSIFEENLF"
sEH_seq = 'MTLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPEGATTRLMKGEITLSQWIPLMEENCRKCSETAKVCLPKNFSIKEIFDKAISARKINRPMLQAALMLRKKGFTTAILTNTWLDDRAERDGLAQLMCELKMHFDFLIESCQVGMVKPEPQIYKFLLDTLKASPSEVVFLDDIGANLKPARDLGMVTILVQDTDTALKELEKVTGIQLLNTPAPLPTSCNPSDMSHGYVTVKPRVRLHFVELGSGPAVCLCHGFPESWYSWRYQIPALAQAGYRVLAMDMKGYGESSAPPEIEEYCMEVLCKEMVTFLDKLGLSQAVFIGHDWGGMLVWYMALFYPERVRAVASLNTPFIPANPNMSPLESIKANPVFDYQLYFQEPGVAEAELEQNLSRTFKSLFRASDESVLSMHKVCEAGGLFVNSPEEPSLSRMVTEEEIQFYVQQFKKSGFRGPLNWYRNMERNWKWACKSLGRKILIPALMVTAEKDFVLVPQMSQHMEDWIPHLKRGHIEDCGHWTQMDKPTEVNQILIKWLDSDARNPPVVSKM'
HSA_seq = 'MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIARRHPYFYAPELLFFAKRYKAAFTECCQAADKAACLLPKLDELRDEGKASSAKQRLKCASLQKFGERAFKAWAVARLSQRFPKAEFAEVSKLVTDLTKVHTECCHGDLLECADDRADLAKYICENQDSISSKLKECCEKPLLEKSHCIAEVENDEMPADLPSLAADFVESKDVCKNYAEAKDVFLGMFLYEYARRHPDYSVVLLLRLAKTYETTLEKCCAAADPHECYAKVFDEFKPLVEEPQNLIKQNCELFEQLGEYKFQNALLVRYTKKVPQVSTPTLVEVSRNLGKVGSKCCKHPEAKRMPCAEDYLSVVLNQLCVLHEKTPVSDRVTKCCTESLVNRRPCFSALEVDETYVPKEFNAETFTFHADICTLSEKERQIKKQTALVELVKHKPKATKEQLKAVMDDFAAFVEKCCKADDKETCFAEEGKKLVAASQAALGL'

In [ ]:
train['protein_sequence'] = train['protein_sequence'].replace({'BRD4': BRD4_seq, 'sEH': sEH_seq, 'HSA': HSA_seq})
train.head()

,drug_smiles,protein_sequence,binding
0,O=C(NCCNc1nc(NC[C@]2(CO)COC[C@H]3CCCN32)nc(N[C...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
1,Cn1nnc(Nc2nc(Nc3ccc(CC4COC(=O)N4)cc3)nc(N[C@H]...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
2,COc1ccc(Nc2nc(NCCNC(N)=O)nc(N(C)[C@@H](CC3CCCC...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
3,COc1ccc(Nc2nc(Nc3ncc(C)s3)nc(N3CSC[C@H]3C(=O)N...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
4,COCc1ccccc1CNc1nc(NCc2cncc(F)c2)nc(N[C@H](Cc2c...,MTLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGP...,0


In [ ]:
# prompt: generate a sample of 100,000 observation from train, half having binding value as 0 and the other half as 1

pos_train = train[train['binding'] == 1].sample(n=50000, random_state=42)
neg_train = train[train['binding'] == 0].sample(n=50000, random_state=42)
train_sample = pd.concat([pos_train, neg_train], axis=0)

In [ ]:
def encode_protein_sequence(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    encoder = LabelEncoder().fit(list(amino_acids))
    return encoder.transform(list(sequence))
train_sample['encoded_protein'] = train_sample['protein_sequence'].apply(encode_protein_sequence)

,drug_smiles,protein_sequence,binding
0,O=C(NCCNc1nc(NC[C@]2(CO)COC[C@H]3CCCN32)nc(N[C...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
1,Cn1nnc(Nc2nc(Nc3ccc(CC4COC(=O)N4)cc3)nc(N[C@H]...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
2,COc1ccc(Nc2nc(NCCNC(N)=O)nc(N(C)[C@@H](CC3CCCC...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
3,COc1ccc(Nc2nc(Nc3ncc(C)s3)nc(N3CSC[C@H]3C(=O)N...,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPP...,0
4,COCc1ccccc1CNc1nc(NCc2cncc(F)c2)nc(N[C@H](Cc2c...,MTLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGP...,0


In [ ]:
train_sample.head()

,drug_smiles,protein_sequence,binding,encoded_protein
288683,CSc1nnc(CNc2nc(Nc3ccc(N4CCC=C(N5CCOCC5)C4=O)cc...,MTLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGP...,1,"[10, 16, 9, 14, 0, 0, 17, 4, 2, 9, 2, 5, 17, 9..."
314939,CC(CNc1nc(NC[C@@H]2CCO[C@H]2c2cn[nH]c2)nc(N[C@...,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,1,"[10, 8, 18, 17, 16, 4, 7, 15, 9, 9, 4, 9, 4, 1..."
253954,COC(=O)Cc1nc(Nc2nc(Nc3cc(Cl)c([N+](=O)[O-])cn3...,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,1,"[10, 8, 18, 17, 16, 4, 7, 15, 9, 9, 4, 9, 4, 1..."
370374,CC(CNc1nc(NCCc2nccn2C(F)F)nc(N[C@@H](CC(=O)N[D...,MTLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGP...,1,"[10, 16, 9, 14, 0, 0, 17, 4, 2, 9, 2, 5, 17, 9..."
422861,COc1cc(C(=O)N[Dy])c(Nc2nc(NCC=Cc3cccnc3)nc(Nc3...,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,1,"[10, 8, 18, 17, 16, 4, 7, 15, 9, 9, 4, 9, 4, 1..."


In [ ]:
#Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# Convert molecule SMILES to RDKit and obtain ECFP
train_sample['molecule'] = train_sample['drug_smiles'].apply(Chem.MolFromSmiles)
train_sample['ecfp'] = train_sample['molecule'].apply(generate_ecfp)

Streaming output truncated to the last 5000 lines.
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:07:16] DEPRECATION WARNING: please use MorganGenerator
[02:0

In [ ]:
# Padding sequences to ensure consistent input size
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_protein_length = max(train_sample['encoded_protein'].apply(len))
max_drug_length = max(train_sample['ecfp'].apply(len))

X_proteins = pad_sequences(train_sample['encoded_protein'], maxlen=max_protein_length)
X_drugs = pad_sequences(train_sample['ecfp'], maxlen=max_drug_length)
y = train_sample['binding'].values

# Split the data
X_train_proteins, X_test_proteins, X_train_drugs, X_test_drugs, y_train, y_test = train_test_split(
    X_proteins, X_drugs, y, test_size=0.2, random_state=42
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Flatten, Dense, Concatenate

def create_combined_cnn_model(protein_length, drug_length, learning_rate=0.000001):
    # Protein branch
    protein_input = Input(shape=(protein_length, 1))
    protein_conv1 = Conv1D(128, 5, padding='same', activation='relu')(protein_input)
    protein_bn1 = BatchNormalization()(protein_conv1)
    protein_dropout1 = Dropout(0.23)(protein_bn1)
    protein_conv2 = Conv1D(128, 3, padding='same', activation='relu')(protein_dropout1)
    protein_bn2 = BatchNormalization()(protein_conv2)
    protein_dropout2 = Dropout(0.21)(protein_bn2)
    protein_conv3 = Conv1D(64, 3, padding='same', activation='relu')(protein_dropout2)
    protein_bn3 = BatchNormalization()(protein_conv3)
    protein_dropout3 = Dropout(0.47)(protein_bn3)
    protein_flat = Flatten()(protein_dropout3)

    # Drug branch
    drug_input = Input(shape=(drug_length, 1))
    drug_conv1 = Conv1D(128, 5, padding='same', activation='relu')(drug_input)
    drug_bn1 = BatchNormalization()(drug_conv1)
    drug_dropout1 = Dropout(0.23)(drug_bn1)
    drug_conv2 = Conv1D(128, 3, padding='same', activation='relu')(drug_dropout1)
    drug_bn2 = BatchNormalization()(drug_conv2)
    drug_dropout2 = Dropout(0.21)(drug_bn2)
    drug_conv3 = Conv1D(64, 3, padding='same', activation='relu')(drug_dropout2)
    drug_bn3 = BatchNormalization()(drug_conv3)
    drug_dropout3 = Dropout(0.47)(drug_bn3)
    drug_flat = Flatten()(drug_dropout3)

    # Concatenate branches
    combined = Concatenate()([protein_flat, drug_flat])
    dense1 = Dense(128, activation='relu')(combined)
    dense2 = Dense(32, activation='relu')(dense1)
    output = Dense(1, activation='sigmoid')(dense2)

    # Compile model with a specific learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model = Model(inputs=[protein_input, drug_input], outputs=output)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

    return model

# Assuming X_train_proteins and X_train_drugs have been preprocessed and shaped correctly
protein_length = X_train_proteins.shape[1]
drug_length = X_train_drugs.shape[1]

cnn_model = create_combined_cnn_model(protein_length, drug_length, learning_rate=0.000001)
cnn_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 1362, 1)]            0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 1024, 1)]            0         []                            
                                                                                                  
 conv1d_4 (Conv1D)           (None, 1362, 128)            768       ['input_5[0][0]']             
                                                                                                  
 conv1d_7 (Conv1D)           (None, 1024, 128)            768       ['input_6[0][0]']             
                                                                                            

In [ ]:
history = model.fit(
    [X_train_proteins, X_train_drugs], y_train,
    validation_split=0.2,
    epochs=9,
    batch_size=32
)

Epoch 1/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1449 - accuracy: 0.9416 - val_loss: 0.2358 - val_accuracy: 0.9179
Epoch 2/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1373 - accuracy: 0.9450 - val_loss: 0.2550 - val_accuracy: 0.9149
Epoch 3/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1303 - accuracy: 0.9477 - val_loss: 0.2607 - val_accuracy: 0.9158
Epoch 4/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1247 - accuracy: 0.9501 - val_loss: 0.2597 - val_accuracy: 0.9159
Epoch 5/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1190 - accuracy: 0.9524 - val_loss: 0.2544 - val_accuracy: 0.9140
Epoch 6/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1145 - accuracy: 0.9539 - val_loss: 0.2662 - val_accuracy: 0.9137
Epoch 7/9
2000/2000 [==============================] - 40s 20ms/step - loss: 0.1079 - accuracy: 0.9569 - val_loss: 0.2986 - val_ac

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([X_test_proteins, X_test_drugs], y_test)
print(f'Test Accuracy: {accuracy}')


625/625 [==============================] - 2s 3ms/step - loss: 0.2739 - accuracy: 0.9143
Test Accuracy: 0.9143499732017517
